In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# 생성자 모델 만들기
generator = Sequential()
# 7x7x128 크기의 feature map을 생성
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))
generator.add(BatchNormalization())
generator.add(Reshape((7, 7, 128)))
generator.add(UpSampling2D())
generator.add(Conv2D(64, kernel_size=5, padding='same'))
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))


In [ ]:
# 판별자 모델 만들기
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False

In [ ]:
# 생성자와 판별자 연결
ginput = Input(shape=(100,))
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()

In [ ]:
# 학습 함수 정의
def gan_train(epoch, batch_size, saving_interval):
    (X_train, _), (_, _) = mnist.load_data()
    X_train = X_train.reshape(-1, 28, 28, 1).astype('float32')
    X_train = (X_train - 127.5) / 127.5  # 원래 mnist 데이터 값의 범위인 0에서 255 사이의 값을 -1부터 1 사이로 바꿔줌

    true = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for i in range(epoch):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        d_loss_real = discriminator.train_on_batch(imgs, true)

        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)

        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        g_loss = gan.train_on_batch(noise, true)  # 노이즈를 넣고 이게 진짜야 라고 속여야 한다

        print('epoch:%d' % i, ' d_loss:%.4f' % d_loss, ' g_loss:%.4f' % g_loss)

        if i % saving_interval == 0:
            noise = np.random.normal(0, 1, (25, 100))
            gen_imgs = generator.predict(noise)
            gen_imgs = 0.5 * gen_imgs + 0.5

            fig, axs = plt.subplots(5, 5)
            count = 0
            for j in range(5):
                for k in range(5):
                    axs[j, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
                    axs[j, k].axis('off')
                    count += 1
            fig.savefig("gan_mnist_%d.png" % i)

# 학습 실행
gan_train(2001, 32, 200)

In [ ]:
# 모드 붕괴 -> 생성자와 감별자 모델의 성능이 같이 올라가야 좋은 결과가 잘 나오는데 지금 위 코드는 생성자와 감별자 성능이 둘 다 똑같이 안 올라가서 문제다

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
(X_train, _), (_, _) = fashion_mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32')
X_train = (X_train - 127.5) / 127.5

In [ ]:
from tensorflow.keras.datasets import cifar10
(X_train, _), (_, _) = cifar10.load_data()
X_train = X_train.astype('float32')
X_train = (X_train - 127.5) / 127.5

In [ ]:
tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)